# Golden Cross Scanner & Backtester — Optimized with Multiple Confirmations

This notebook implements a golden-cross trading strategy with additional confirmation signals and a simple random-search optimizer to tune parameters for maximizing return or minimizing drawdown. 

It uses price data from Yahoo Finance via `yfinance` and includes extra indicators such as ADX, Stochastic RSI, CCI, SMA slope, and a volume percentile filter. The optimizer samples random parameter combinations and evaluates performance based on a chosen objective metric (CAGR, Sharpe ratio, Sortino ratio, or maximum drawdown).


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import timedelta

# Configuration: tickers, date range, timeframe
TICKERS = "RELIANCE.NS", "TCS.NS""
START_DATE = '2015-01-01'
END_DATE = None  # None means up to today
TIMEFRAME = '1d'

# Confirmation types available: choose subset or use 'k-of-n'
# Example: ['MACD', 'RSI', 'ADX', 'STOCH_RSI', 'CCI', 'SMA_SLOPE', 'VOLUME_PCT']
CONFIRMATION_TYPES = ['MACD', 'RSI']
CONFIRMATION_MODE = 'k-of-n'  # 'all', 'any', or 'k-of-n'
CONFIRMATION_MIN = 1  # used when CONFIRMATION_MODE == 'k-of-n'

# Backtest settings
ATR_PERIOD = 14  # for trailing stop
ATR_MULTIPLIER = 3.0
MAX_HOLD_DAYS = 60  # maximum days to hold a position

# Optimization settings
N_ITERATIONS = 20  # number of random parameter sets to evaluate
OPT_OBJECTIVE = 'cagr'  # objective metric: 'cagr', 'sharpe', 'sortino', 'min_maxdd'


In [10]:
def sma(series: pd.Series, window: int) -> pd.Series:
    return series.rolling(window=window, min_periods=window).mean()

def atr(df: pd.DataFrame, period: int) -> pd.Series:
    high_low = df['High'] - df['Low']
    high_close = np.abs(df['High'] - df['Close'].shift())
    low_close = np.abs(df['Low'] - df['Close'].shift())
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    return tr.rolling(window=period, min_periods=period).mean()

def rsi(series: pd.Series, period: int) -> pd.Series:
    delta = series.diff()
    up = delta.clip(lower=0)
    down = -delta.clip(upper=0)
    ma_up = up.ewm(span=period, adjust=False).mean()
    ma_down = down.ewm(span=period, adjust=False).mean()
    rs = ma_up / ma_down
    return 100 - (100 / (1 + rs))

def macd(series: pd.Series, fast: int = 12, slow: int = 26, signal: int = 9):
    ema_fast = series.ewm(span=fast, adjust=False).mean()
    ema_slow = series.ewm(span=slow, adjust=False).mean()
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    return macd_line, signal_line

def adx(df: pd.DataFrame, period: int = 14):
    # Directional movement
    up_move = df['High'].diff()
    down_move = df['Low'].diff() * -1
    plus_dm = np.where((up_move > down_move) & (up_move > 0), up_move, 0.0)
    minus_dm = np.where((down_move > up_move) & (down_move > 0), down_move, 0.0)
    tr = atr(df, 1)
    atr14 = tr.rolling(window=period).mean()
    plus_di = 100 * pd.Series(plus_dm).rolling(window=period).sum() / atr14
    minus_di = 100 * pd.Series(minus_dm).rolling(window=period).sum() / atr14
    dx = 100 * (plus_di - minus_di).abs() / (plus_di + minus_di)
    adx = dx.rolling(window=period).mean()
    return adx

def stoch_rsi(series: pd.Series, period: int = 14):
    rsi_series = rsi(series, period)
    stoch = (rsi_series - rsi_series.rolling(period).min()) / (rsi_series.rolling(period).max() - rsi_series.rolling(period).min())
    return stoch

def cci(df: pd.DataFrame, period: int = 20):
    tp = (df['High'] + df['Low'] + df['Close']) / 3
    sma_tp = tp.rolling(window=period, min_periods=period).mean()
    mad = (tp - sma_tp).abs().rolling(window=period, min_periods=period).mean()
    cci = (tp - sma_tp) / (0.015 * mad)
    return cci

def sma_slope(series: pd.Series, window: int = 50):
    ma = sma(series, window)
    slope = ma.diff()
    return slope

def volume_percentile(df: pd.DataFrame, window: int = 50):
    vol = df['Volume']
    return vol.rank(pct=True)


In [11]:
def fetch_data(ticker: str, start: str, end: str = None, interval: str = '1d') -> pd.DataFrame:
    df = yf.download(ticker, start=start, end=end, interval=interval, progress=False, multi_level_index=False)
    df = df.dropna()
    return df


In [12]:
def detect_signals(df: pd.DataFrame,
                    fast_window: int = 50, slow_window: int = 200,
                    conf_types: list = None,
                    conf_mode: str = 'k-of-n',
                    conf_min: int = 1) -> pd.DataFrame:
    if conf_types is None:
        conf_types = ['MACD', 'RSI']
    df = df.copy()
    df['fast_sma'] = sma(df['Close'], fast_window)
    df['slow_sma'] = sma(df['Close'], slow_window)
    # Golden cross: fast SMA crossing above slow SMA
    df['gc_signal'] = (df['fast_sma'].shift(1) < df['slow_sma'].shift(1)) & (df['fast_sma'] >= df['slow_sma'])
    # Additional indicators
    if 'RSI' in conf_types:
        df['rsi'] = rsi(df['Close'], 14)
        df['rsi_conf'] = df['rsi'] < 30  # oversold indicates potential buy
    if 'MACD' in conf_types:
        macd_line, signal_line = macd(df['Close'])
        df['macd'] = macd_line
        df['macd_signal'] = signal_line
        df['macd_conf'] = macd_line > signal_line
    if 'ADX' in conf_types:
        df['adx'] = adx(df)
        df['adx_conf'] = df['adx'] > 20
    if 'STOCH_RSI' in conf_types:
        df['stoch_rsi'] = stoch_rsi(df['Close'])
        df['stoch_conf'] = df['stoch_rsi'] < 0.2
    if 'CCI' in conf_types:
        df['cci'] = cci(df)
        df['cci_conf'] = df['cci'] < -100
    if 'SMA_SLOPE' in conf_types:
        df['sma_slope'] = sma_slope(df['Close'])
        df['slope_conf'] = df['sma_slope'] > 0
    if 'VOLUME_PCT' in conf_types:
        df['vol_pct'] = volume_percentile(df)
        df['vol_conf'] = df['vol_pct'] > 0.5
    # Determine confirmation
    conf_columns = [c for c in df.columns if c.endswith('_conf')]
    df['conf_count'] = df[conf_columns].sum(axis=1)
    if conf_mode == 'all':
        df['is_confirmed'] = df['conf_count'] == len(conf_columns)
    elif conf_mode == 'any':
        df['is_confirmed'] = df['conf_count'] > 0
    else:  # k-of-n
        df['is_confirmed'] = df['conf_count'] >= conf_min
    # Final entry signals: golden cross + confirmation
    df['entry_signal'] = df['gc_signal'] & df['is_confirmed']
    return df


In [13]:
def backtest(df: pd.DataFrame,
             atr_period: int = 14,
             atr_mult: float = 3.0,
             max_hold: int = 60) -> tuple:
    # df must include 'entry_signal', 'fast_sma', 'slow_sma'
    cash = 1.0  # normalized capital
    position = 0.0
    buy_price = 0.0
    hold_days = 0
    equity = []
    trade_log = []
    df = df.copy()
    df['ATR'] = atr(df, atr_period)
    for date, row in df.iterrows():
        price = row['Close']
        if position == 0:
            # check entry
            if row['entry_signal']:
                position = cash / price
                buy_price = price
                cash = 0
                hold_days = 0
                trade_log.append({'entry_date': date, 'entry_price': price})
        else:
            hold_days += 1
            exit_reason = None
            # exit conditions
            if row['fast_sma'] < row['slow_sma']:
                exit_reason = 'death_cross'
            elif row['ATR'] > 0 and price < buy_price - atr_mult * row['ATR']:
                exit_reason = 'atr_stop'
            elif hold_days >= max_hold:
                exit_reason = 'max_hold'
            if exit_reason:
                cash = position * price
                trade_log[-1].update({'exit_date': date, 'exit_price': price, 'reason': exit_reason})
                position = 0
                buy_price = 0
                hold_days = 0
        # record daily equity
        equity_val = cash + position * price
        equity.append({'date': date, 'equity': equity_val})
    equity_df = pd.DataFrame(equity).set_index('date')
    trades_df = pd.DataFrame(trade_log)
    return equity_df, trades_df


In [14]:
def compute_metrics(equity_df: pd.DataFrame) -> dict:
    returns = equity_df['equity'].pct_change().dropna()
    if len(returns) == 0:
        return {'cagr': 0, 'sharpe': 0, 'sortino': 0, 'max_dd': 0}
    cumulative_return = equity_df['equity'].iloc[-1] / equity_df['equity'].iloc[0] - 1
    num_years = (equity_df.index[-1] - equity_df.index[0]).days / 365.25
    cagr = (1 + cumulative_return) ** (1 / num_years) - 1 if num_years > 0 else 0
    sharpe = np.sqrt(252) * returns.mean() / returns.std() if returns.std() != 0 else 0
    downside_returns = returns[returns < 0]
    sortino = np.sqrt(252) * returns.mean() / downside_returns.std() if downside_returns.std() != 0 else 0
    # max drawdown
    cumulative = (1 + returns).cumprod()
    peak = cumulative.cummax()
    dd = (cumulative - peak) / peak
    max_dd = dd.min()
    return {'cagr': cagr, 'sharpe': sharpe, 'sortino': sortino, 'max_dd': max_dd}


In [15]:
import random

def optimize_parameters(data: pd.DataFrame,
                        n_iter: int = N_ITERATIONS,
                        objective: str = OPT_OBJECTIVE,
                        conf_types: list = CONFIRMATION_TYPES,
                        conf_mode: str = CONFIRMATION_MODE,
                        conf_min: int = CONFIRMATION_MIN) -> dict:
    best_score = -np.inf
    best_params = None
    best_metrics = None
    for i in range(n_iter):
        # Randomly choose parameters within reasonable ranges
        fast_window = random.randint(20, 100)
        slow_window = random.randint(fast_window + 20, fast_window + 200)
        atr_period = random.randint(10, 30)
        atr_mult = round(random.uniform(2.0, 4.0), 2)
        max_hold = random.randint(30, 120)
        conf_min_local = random.randint(1, len(conf_types)) if conf_mode == 'k-of-n' else 1
        # Detect signals
        signals = detect_signals(data, fast_window, slow_window, conf_types, conf_mode, conf_min_local)
        # Backtest
        equity_df, trades_df = backtest(signals, atr_period, atr_mult, max_hold)
        metrics = compute_metrics(equity_df)
        # Determine objective score
        if objective == 'cagr':
            score = metrics['cagr']
        elif objective == 'sharpe':
            score = metrics['sharpe']
        elif objective == 'sortino':
            score = metrics['sortino']
        elif objective == 'min_maxdd':
            score = -metrics['max_dd']
        else:
            score = metrics['cagr']
        # Update best
        if score > best_score:
            best_score = score
            best_params = {
                'fast_window': fast_window,
                'slow_window': slow_window,
                'atr_period': atr_period,
                'atr_mult': atr_mult,
                'max_hold': max_hold,
                'conf_min': conf_min_local
            }
            best_metrics = metrics
    return best_params, best_metrics


In [16]:
# Fetch data for a single ticker for optimization
selected_ticker = TICKERS[0]
data = fetch_data(selected_ticker, START_DATE, END_DATE, TIMEFRAME)

# Optimize parameters on this single ticker's data
best_params, best_metrics = optimize_parameters(data)
print('Best parameters:', best_params)
print('Performance metrics:', best_metrics)

# Apply best parameters to all tickers and backtest
all_equity = None
all_trades = []
for ticker in TICKERS:
    df = fetch_data(ticker, START_DATE, END_DATE, TIMEFRAME)
    signals = detect_signals(df,
                             best_params['fast_window'], best_params['slow_window'],
                             CONFIRMATION_TYPES, CONFIRMATION_MODE, best_params['conf_min'])
    equity_df, trades_df = backtest(signals,
                                    best_params['atr_period'], best_params['atr_mult'], best_params['max_hold'])
    equity_df['ticker'] = ticker
    trades_df['ticker'] = ticker
    if all_equity is None:
        all_equity = equity_df
    else:
        all_equity = all_equity.join(equity_df['equity'], how='outer', rsuffix=f'_{ticker}')
    all_trades.append(trades_df)

# Combine trades and save results
all_trades_df = pd.concat(all_trades, ignore_index=True)
all_trades_df.to_csv('gc_trades_optimized.csv', index=False)
all_equity.to_csv('gc_equity_optimized.csv')

# Save best parameters to JSON
import json
with open('gc_best_params.json', 'w') as f:
    json.dump({'params': best_params, 'metrics': best_metrics}, f, indent=2)

print('Optimization complete. Files saved: gc_trades_optimized.csv, gc_equity_optimized.csv, gc_best_params.json')


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_5872/3272344764.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end, interval=interval, progress=False, multi_level_index=False)


Best parameters: {'fast_window': 31, 'slow_window': 194, 'atr_period': 30, 'atr_mult': 3.78, 'max_hold': 52, 'conf_min': 1}
Performance metrics: {'cagr': np.float64(0.022885791254654064), 'sharpe': np.float64(0.32357668276055196), 'sortino': np.float64(0.19497478537983837), 'max_dd': np.float64(-0.22160458954252102)}


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_5872/3272344764.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end, interval=interval, progress=False, multi_level_index=False)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_5872/3272344764.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end, interval=interval, progress=False, multi_level_index=False)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_5872/3272344764.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end, interval=interval, progress=False, multi_level_index=False)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_5872/3272344764.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=star

Optimization complete. Files saved: gc_trades_optimized.csv, gc_equity_optimized.csv, gc_best_params.json
